# Nonogram (数织)

Nonograms, also known as Hanjie, Paint by Numbers, Picross, Griddlers, and Pic-a-Pix are picture logic puzzles in which cells in a grid must be colored or left blank according to numbers at the edges of the grid to reveal a hidden picture. In this puzzle, the numbers are a form of discrete tomography that measures how many unbroken lines of filled-in squares there are in any given row or column. For example, a clue of "4 8 3" would mean there are sets of four, eight, and three filled squares, in that order, with at least one blank square between successive sets.  

----------

Nonograms，也称为 Hanjie、Paint by Numbers、Picross、Griddlers 和 Pic-a-Pix 是图片逻辑谜题，其中网格中的单元格必须根据网格边缘的数字着色或留空以显示隐藏的图片。在这个谜题中，数字是离散断层扫描的一种形式，用于测量任何给定行或列中有多少条连续的填充方块行。例如，“4 8 3”的线索意味着有四、八和三个实心方块的集合，按该顺序，连续集合之间至少有一个空白方块。


Codes from: https://rosettacode.org/wiki/Nonogram_solver#Python_3. 

And: https://hackerry.github.io/archive/2022/02/28/Solving-Nonogram.html  The authors also provide extensive explanations for this problem. GREAT THANKS! 💗

In [33]:
def readGrid(path):
    with open(f"../assets/data/Nonogram/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        col_constraints = []
        row_constraints = []
        tmp1 = []
        tmp2 = []
        for i in range(int(n)):
            tmp1.append(f.readline())
        for i in range(int(m)):
            tmp2.append(f.readline())
        
        col_constraints = [list(map(int, g.strip().split(" "))) for g in tmp1]
        row_constraints = [list(map(int, g.strip().split(" "))) for g in tmp2]
        return int(m), int(n), col_constraints, row_constraints

if __name__ == "__main__":
    m, n, col_c, row_c = readGrid("256_30x40")
    for g in col_c[:5]:
        print(g)
    print("=====")
    for a in row_c[:5]:
        print(a)
    print(m, n)

[0]
[4]
[5]
[6]
[2, 4]
=====
[4]
[4, 1, 8]
[10, 2, 2, 2, 1, 11]
[10, 1, 1, 1, 3, 4, 2]
[2, 7, 1, 8, 1]
30 40


In [ ]:
# Author: hackerry
# Links: https://hackerry.github.io/archive/2022/02/28/Solving-Nonogram.html

import z3
import time

# Generate all constraints for one dimension
def constraints_one_dim(solver, constraints, other_dim_len, identifier='v'):
    result = []
    
    # List of new constraint
    for line_num in range(len(constraints)):
        last_var = None
        new_vars = []
        
        # Used for computing the improved range constraint
        max_start = other_dim_len - (sum(constraints[line_num]) + len(constraints[line_num]) - 1)
        min_start = 0
        
        # List of new variables
        for span_num in range(len(constraints[line_num])):            
            # For each number, create new var denoting the start index
            new_var = z3.Int("%s_%s_%s" % (identifier, line_num, span_num))
            
            # Regular range constraint
            solver.add(z3.And(new_var >= 0, new_var < other_dim_len))
            
            # Improved range constraint similar to the Simple Box Rule here: https://en.wikipedia.org/wiki/Nonogram
            solver.add(z3.And(new_var >= min_start, new_var <= max_start))
            
            # Order + Gap constraint
            if last_var != None:
                solver.add(last_var+constraints[line_num][span_num-1] < new_var)
            
            min_start = min_start + constraints[line_num][span_num] + 1
            max_start = max_start + constraints[line_num][span_num] + 1
            last_var = new_var
            new_vars.append(new_var)    
        result.append(new_vars)
    return result

# Generate all constraints for each cell
def constraints_cells(solver, row_vars, col_vars, row_len, col_len, col_constraints, row_constraints):
    # Generate all vars for each cell - easy for output, not necessary
    board = []
    for row_num in range(row_len):
        row = [z3.Bool('c_%s_%s' % (row_num, col_num)) for col_num in range(col_len)]
        board.append(row)
    
    # Generate consistency constraints
    for row_num in range(row_len):
        for col_num in range(col_len):
            # Either this cell is empty - no equality exists
            empty_cond_row = [z3.Or(row_vars[row_num][i] > col_num, row_vars[row_num][i] + row_constraints[row_num][i] <= col_num) 
                              for i in range(len(row_vars[row_num]))]
            empty_cond_col = [z3.Or(col_vars[col_num][i] > row_num, col_vars[col_num][i] + col_constraints[col_num][i] <= row_num) 
                              for i in range(len(col_vars[col_num]))]
            empty = z3.And(*empty_cond_row, *empty_cond_col)
            
            # Or this cell is filled - required by both row and col vars
            taken_cond_row = [z3.And(row_vars[row_num][i] <= col_num, row_vars[row_num][i] + row_constraints[row_num][i] > col_num) 
                              for i in range(len(row_vars[row_num]))]
            taken_cond_col = [z3.And(col_vars[col_num][i] <= row_num, col_vars[col_num][i] + col_constraints[col_num][i] > row_num) 
                              for i in range(len(col_vars[col_num]))]
            taken = z3.And(z3.Or(*taken_cond_row), z3.Or(*taken_cond_col))
            
            # Record result
            board[row_num][col_num] = taken
            solver.add(z3.Or(empty, taken))
    return board

def nonogram_solver(X, Y,col_constraints, row_constraints):
    start = time.perf_counter()
    s = z3.Solver()
    row_len = len(row_constraints)
    col_len = len(col_constraints)
    row_vars = constraints_one_dim(s, row_constraints, col_len, 'r')
    col_vars = constraints_one_dim(s, col_constraints, row_len, 'c')
    board = constraints_cells(s, row_vars, col_vars, row_len, col_len, col_constraints, row_constraints)

    if s.check() == z3.sat:
        m = s.model()
        # print(m)
        for row_num in range(row_len):
            for col_num in range(col_len):
                if m.evaluate(board[row_num][col_num]):
                    print('\u2588', end=' ')
                else:
                    print(' ', end=' ')
            print()
    else:
        print("unsat")
    end = time.perf_counter()
    print("Time:", (end-start), "s")

if __name__ == "__main__":
    X, Y, col_c, row_c = readGrid("256_30x40")
    nonogram_solver(X, n, col_c, row_c)

                                                            █ █ █ █             
            █ █ █ █                                 █   █ █ █ █ █ █ █ █         
        █ █ █ █ █ █ █ █ █ █   █ █   █ █   █ █   █   █ █ █ █ █ █ █ █ █ █ █       
      █ █ █ █ █ █ █ █ █ █         █     █     █       █ █ █   █ █ █ █   █ █     
    █ █   █ █ █ █ █ █ █                   █             █ █ █ █ █ █ █ █   █     
    █ █ █ █ █ █ █ █ █                 █   █ █ █ █         █ █ █ █   █ █   █     
  █ █ █ █ █ █ █ █ █ █           █ █   █   █   █ █ █         █ █   █   █ █ █     
  █ █ █ █ █ █ █ █ █   █         █ █         █ █ █ █           █ █ █ █ █   █     
  █ █ █ █ █ █ █ █   █       █ █ █ █ █       █ █ █ █ █ █ █       █ █             
  █       █ █ █   █     █ █ █ █ █ █ █ █       █ █ █ █ █ █ █ █   █ █             
            █ █   █ █ █ █ █ █ █ █ █ █ █       █ █ █ █   █ █ █   █               
              █ █ █ █ █ █ █ █   █ █ █           █ █ █ █ █ █   █                 
                █       █ █ 

In [35]:
from functools import reduce

def gen_row(w, s):
    """Create all patterns of a row or col that match given runs."""
    def gen_seg(o, sp):
        if not o:
            return [[2] * sp]
        return [[2] * x + o[0] + tail
                for x in range(1, sp - len(o) + 2)
                for tail in gen_seg(o[1:], sp - x)]
 
    return [x[1:] for x in gen_seg([[1] * i for i in s], w + 1 - sum(s))]
 
 
def deduce(hr, vr):
    """Fix inevitable value of cells, and propagate."""
    def allowable(row):
        return reduce(lambda a, b: [x | y for x, y in zip(a, b)], row)
 
    def fits(a, b):
        return all(x & y for x, y in zip(a, b))
 
    def fix_col(n):
        """See if any value in a given column is fixed;
        if so, mark its corresponding row for future fixup.
        
        
        """
        c = [x[n] for x in can_do]
        cols[n] = [x for x in cols[n] if fits(x, c)]
        for i, x in enumerate(allowable(cols[n])):
            if x != can_do[i][n]:
                mod_rows.add(i)
                can_do[i][n] &= x
 
    def fix_row(n):
        """Ditto, for rows."""
        c = can_do[n]
        rows[n] = [x for x in rows[n] if fits(x, c)]
        for i, x in enumerate(allowable(rows[n])):
            if x != can_do[n][i]:
                mod_cols.add(i)
                can_do[n][i] &= x
 
    def show_gram(m):
        # If there's 'x', something is wrong.
        # If there's '?', needs more work.
        for x in m:
            print(" ".join("x#.?"[i] for i in x))
        print()
 
    w, h = len(vr), len(hr)
    rows = [gen_row(w, x) for x in hr]
    cols = [gen_row(h, x) for x in vr]
    can_do = list(map(allowable, rows))
 
    # Initially mark all columns for update.
    mod_rows, mod_cols = set(), set(range(w))
 
    while mod_cols:
        for i in mod_cols:
            fix_col(i)
        mod_cols = set()
        for i in mod_rows:
            fix_row(i)
        mod_rows = set()
 
    if all(can_do[i][j] in (1, 2) for j in range(w) for i in range(h)):
        print("Solution would be unique")  # but could be incorrect!
    else:
        print("Solution may not be unique, doing exhaustive search:")
 
    # We actually do exhaustive search anyway. Unique solution takes
    # no time in this phase anyway, but just in case there's no
    # solution (could happen?).
    out = [0] * h
 
    def try_all(n = 0):
        if n >= h:
            for j in range(w):
                if [x[j] for x in out] not in cols[j]:
                    return 0
            # show_gram(out)
            return 1
        sol = 0
        for x in rows[n]:
            out[n] = x
            sol += try_all(n + 1)
        return sol
 
    n = try_all()
    if not n:
        print("No solution.")
    elif n == 1:
        print("Unique solution.")
    else:
        print(n, "solutions.")
    print()


def solve(row_constraints, col_constraints, show_runs=True):
    if show_runs:
        print("Horizontal runs:", row_constraints)
        print("Vertical runs:", col_constraints)
    deduce(row_constraints, col_constraints)

# def solve(s, show_runs=True):
#     s = [[[ord(c) - ord('A') + 1 for c in w] for w in l.split()]
#          for l in s.splitlines()]
#     if show_runs:
#         print("Horizontal runs:", s[0])
#         print("Vertical runs:", s[1])
#     deduce(s[0], s[1])

if __name__ == "__main__":
#     solve("""E BCB BEA BH BEK AABAF ABAC BAA BFB OD JH BADCF Q Q R AN AAN EI H G
# E CB BAB AAA AAA AC BB ACC ACCA AGB AIA AJ AJ ACE AH BAF CAG DAG FAH FJ GJ ADK ABK BL CM""")
    X, Y, col_constraints, row_constraints = readGrid("256_30x40")
    solve(row_constraints, col_constraints)

Horizontal runs: [[4], [4, 1, 8], [10, 2, 2, 2, 1, 11], [10, 1, 1, 1, 3, 4, 2], [2, 7, 1, 8, 1], [9, 1, 4, 4, 2, 1], [10, 2, 1, 1, 3, 2, 1, 3], [9, 1, 2, 4, 5, 1], [8, 1, 5, 7, 2], [1, 3, 1, 8, 8, 2], [2, 11, 4, 3, 1], [8, 3, 6, 1], [1, 6, 6], [1, 7, 3, 4], [1, 5, 5, 1, 1], [2, 2, 1, 6, 2, 1], [1, 4, 9, 1, 1], [2, 1, 2, 9, 3], [8, 7, 2], [2, 5, 2, 5, 2, 2], [3, 4, 10, 3], [4, 3, 8, 2, 1, 1], [4, 3, 7, 1], [8, 1, 2, 3, 1, 1], [6, 4, 5, 1], [6, 5, 2, 2, 1], [1, 7, 2, 1, 4], [2, 4, 1, 5], [0], [0]]
Vertical runs: [[0], [4], [5], [6], [2, 4], [8], [10], [11], [8, 5, 7], [7, 3, 11], [5, 1, 2, 1, 7], [3, 1, 2, 1, 2, 5, 1], [2, 12, 5], [1, 8, 9], [7, 12], [1, 3, 12, 2], [1, 9, 3], [1, 8, 1, 1, 4], [1, 4, 2, 3, 4], [1, 2, 2, 2, 3, 2], [1, 12, 1], [1, 3, 11, 1], [1, 1, 2, 10, 3], [1, 6, 13], [1, 7, 11, 2], [7, 4, 1, 4, 1], [2, 6, 6, 4], [2, 2, 5, 3, 1, 2], [4, 6, 3, 2], [5, 6, 5, 1], [3, 3, 2, 1, 6, 2, 1], [8, 1, 2, 1], [6, 4], [5, 4], [5, 1], [2, 4], [2, 1], [5], [0], [0]]
Solution would be un

In [31]:
import z3
import time

# Generate all constraints for one dimension
def constraints_one_dim(solver, constraints, other_dim_len, identifier='v'):
    result = []
    
    # List of new constraint
    for line_num in range(len(constraints)):
        last_var = None
        new_vars = []
        
        # Used for computing the improved range constraint
        max_start = other_dim_len - (sum(constraints[line_num]) + len(constraints[line_num]) - 1)
        min_start = 0
        
        # List of new variables
        for span_num in range(len(constraints[line_num])):            
            # For each number, create new var denoting the start index
            new_var = z3.Int("%s_%s_%s" % (identifier, line_num, span_num))
            
            # Regular range constraint
            solver.add(z3.And(new_var >= 0, new_var < other_dim_len))
            
            # Improved range constraint similar to the Simple Box Rule here: https://en.wikipedia.org/wiki/Nonogram
            solver.add(z3.And(new_var >= min_start, new_var <= max_start))
            
            # Order + Gap constraint
            if last_var != None:
                solver.add(last_var+constraints[line_num][span_num-1] < new_var)
            
            min_start = min_start + constraints[line_num][span_num] + 1
            max_start = max_start + constraints[line_num][span_num] + 1
            last_var = new_var
            new_vars.append(new_var)    
        result.append(new_vars)
    return result

# Generate all constraints for each cell
def constraints_cells(solver, row_vars, col_vars, row_len, col_len):
    # Generate all vars for each cell - easy for output, not necessary
    board = []
    for row_num in range(row_len):
        row = [z3.Bool('c_%s_%s' % (row_num, col_num)) for col_num in range(col_len)]
        board.append(row)
    
    # Generate consistency constraints
    for row_num in range(row_len):
        for col_num in range(col_len):
            # Either this cell is empty - no equality exists
            empty_cond_row = [z3.Or(row_vars[row_num][i] > col_num, row_vars[row_num][i] + row_constraints[row_num][i] <= col_num) 
                              for i in range(len(row_vars[row_num]))]
            empty_cond_col = [z3.Or(col_vars[col_num][i] > row_num, col_vars[col_num][i] + col_constraints[col_num][i] <= row_num) 
                              for i in range(len(col_vars[col_num]))]
            empty = z3.And(*empty_cond_row, *empty_cond_col)
            
            # Or this cell is filled - required by both row and col vars
            taken_cond_row = [z3.And(row_vars[row_num][i] <= col_num, row_vars[row_num][i] + row_constraints[row_num][i] > col_num) 
                              for i in range(len(row_vars[row_num]))]
            taken_cond_col = [z3.And(col_vars[col_num][i] <= row_num, col_vars[col_num][i] + col_constraints[col_num][i] > row_num) 
                              for i in range(len(col_vars[col_num]))]
            taken = z3.And(z3.Or(*taken_cond_row), z3.Or(*taken_cond_col))
            
            # Record result
            board[row_num][col_num] = taken
            solver.add(z3.Or(empty, taken))
    return board

start = time.perf_counter()
s = z3.Solver()

row_constraints = row_c
col_constraints = col_c
row_len = len(row_constraints)
col_len = len(col_constraints)
row_vars = constraints_one_dim(s, row_constraints, col_len, 'r')
col_vars = constraints_one_dim(s, col_constraints, row_len, 'c')
board = constraints_cells(s, row_vars, col_vars, row_len, col_len)

if s.check() == z3.sat:
    m = s.model()
    # print(m)
    for row_num in range(row_len):
        for col_num in range(col_len):
            if m.evaluate(board[row_num][col_num]):
                print('\u2588', end=' ')
            else:
                print(' ', end=' ')
        print()
else:
    print("unsat")
end = time.perf_counter()
print("Time:", (end-start), "s")

                                                            █ █ █ █             
            █ █ █ █                                 █   █ █ █ █ █ █ █ █         
        █ █ █ █ █ █ █ █ █ █   █ █   █ █   █ █   █   █ █ █ █ █ █ █ █ █ █ █       
      █ █ █ █ █ █ █ █ █ █         █     █     █       █ █ █   █ █ █ █   █ █     
    █ █   █ █ █ █ █ █ █                   █             █ █ █ █ █ █ █ █   █     
    █ █ █ █ █ █ █ █ █                 █   █ █ █ █         █ █ █ █   █ █   █     
  █ █ █ █ █ █ █ █ █ █           █ █   █   █   █ █ █         █ █   █   █ █ █     
  █ █ █ █ █ █ █ █ █   █         █ █         █ █ █ █           █ █ █ █ █   █     
  █ █ █ █ █ █ █ █   █       █ █ █ █ █       █ █ █ █ █ █ █       █ █             
  █       █ █ █   █     █ █ █ █ █ █ █ █       █ █ █ █ █ █ █ █   █ █             
            █ █   █ █ █ █ █ █ █ █ █ █ █       █ █ █ █   █ █ █   █               
              █ █ █ █ █ █ █ █   █ █ █           █ █ █ █ █ █   █                 
                █       █ █ 